In [1]:
# Vladyslav Cherevkov
# vc1238

import requests
import json
import pandas as pd
from watson_developer_cloud.natural_language_understanding_v1 import Features, EntitiesOptions, KeywordsOptions

In [2]:
articles_urls = []

def getNews(src):
    i=0
    endpoint = 'https://newsapi.org/v2/top-headlines' #might need to change to everything
    parameters = {'apiKey': 'ef18e68a2dfe4494b6dbe053077f4022',
                  'sources': src
                 }
    resp = requests.get(endpoint, params=parameters)
    all_articles = resp.json()
    while(True):
        try:
            articles_urls.append(all_articles['articles'][i]['url']) #only gives 20, API problem or code?
            i=i+1
        except:
            break
    return articles_urls


In [3]:
#part 2 continued

def cleanEntities(source, url, entities):
    new_art_urls = []
    i=0
    while(i<len(entities)):
        new_dict = {'entity':entities[i]['text'],
                    'url': url,
                    'source': source,
                    'relevance': entities[i]['relevance'],
                    'sentiment': entities[i]['sentiment']['score']
               }
        i=i+1
        new_art_urls.append(new_dict)
    return new_art_urls

In [4]:
#part 2

def extractEntities(url, source):
    endpoint_watson = "https://gateway.watsonplatform.net/natural-language-understanding/api/v1/analyze"
    params = {
        'version': '2017-02-27',
    }
    headers = { 
        'Content-Type': 'application/json',
    }
    watson_options = {
      "url": url,
      "features": {
        "entities": {
          "sentiment": True
        }
      }
    }
    username = "2428f61d-be04-4055-9e6f-3356d8cd343f"
    password = "G6BHcIQ5oBmK"

    resp = requests.post(endpoint_watson, 
                         data=json.dumps(watson_options), 
                         headers=headers, 
                         params=params, 
                         auth=(username, password) 
                        )
    responce = resp.json()
    data = responce['entities']
    data = cleanEntities(source, url, responce['entities'])
    return data



In [5]:
source_wsj= 'the-wall-street-journal'
source_nyt= 'the-new-york-times'
wsj_urls = getNews(source_wsj)
nyt_urls = getNews(source_nyt)

entities = []
for i in wsj_urls:
    entities += extractEntities(i,source_wsj)
    
for i in nyt_urls:
    entities += extractEntities(i,source_nyt)
    
df = pd.DataFrame(entities)
df

,entity,relevance,sentiment,source,url
0,Brett Kavanaugh,0.859622,-0.389738,the-wall-street-journal,https://www.wsj.com/articles/kavanaugh-accuser...
1,Christine Blasey Ford,0.666312,0.000000,the-wall-street-journal,https://www.wsj.com/articles/kavanaugh-accuser...
2,Judiciary Committee,0.652850,0.000000,the-wall-street-journal,https://www.wsj.com/articles/kavanaugh-accuser...
3,Supreme Court,0.615370,-0.596478,the-wall-street-journal,https://www.wsj.com/articles/kavanaugh-accuser...
4,Chuck Grassley,0.573541,0.000000,the-wall-street-journal,https://www.wsj.com/articles/kavanaugh-accuser...
5,assault,0.536022,-0.389738,the-wall-street-journal,https://www.wsj.com/articles/kavanaugh-accuser...
6,Chairman,0.493824,0.000000,the-wall-street-journal,https://www.wsj.com/articles/kavanaugh-accuser...
7,Iowa,0.442224,0.265333,the-wall-street-journal,https://www.wsj.com/articles/kavanaugh-accuser...
8,professor,0.419599,0.000000,the-wall-street-journal,https://www.wsj.com/articles/kavanaugh-accuser...
9,California,0.393112,0.000000,the-wall-street-journal,https://www.wsj.com/articles/kavanaugh-accuser...
